## Four Jarvis Auto Workflow Python Demo Script 1

简单的自动数据报告工作流Demo：从网页提取所需数据，自动生成舆情报告，定时邮件发送   

简介：从东方财富网资讯频道，根据自定义关键词（公司名称），通过selenium采集数据，用re正则表达式清洗数据，提取新闻标题、时间、链接、新闻摘要、公司名称一起存入Mysql数据库，再生成数据报告，定时邮件发送

目的：拒绝无脑重复工作

流程：数据采集、数据清洗、数据储存、数据导出、定时邮件

##### 数据采集 Tool: selenium/谷歌浏览器/ChromeDriver

##### 数据清洗 Tool: 正则表达式re

##### 数据储存 Tool: 数据库mysql/pymysql

##### 数据导出 Tool: pymysql/pandas/matplotlib/python-docx/openpyxl/python-pptx

##### 定时邮件 Tool: SMTP/schedule

##### 其他工具 Tool: 多进程multiprocessing.Pool/日志处理loguru/python内置库

##### 实现过程：

def selenium(): 传入url，传回网页源码

def eastmoney2mysql(): 通过正则表达式提取网页数据，并清洗导入mysql数据库

class Fmysql: 自定义mysql常用语句的类，执行sql语句

def get_Ystd(): / def get_Today(): 生成昨天/今天的日期字符串

def save2picture(): 传入日期和文件保存路径，通过pandas的read_sql()方法生成数据，进而通过matplotlib生成图片

def save2excel(): / def save2docx(): / def save2pptx(): 传入日期/公司/文件保存路径，调用Fmysql类，执行sql语句生成数据，写入excel/word/ppt

def create_email(): 传入发件人昵称/收件人昵称/主题/正文/附件地址,附件名称，生成一封邮件

def send_email(): 传入发件人邮箱账号/密码/收件人邮箱/邮件内容，发送邮件

def main_by_schedule(): 自定义定时函数

def main():  实现以上函数，通过multiprocessing开启多进程实现数据采集，最后生成数据报告和发送邮件，再通过自定义定时函数执行main函数



##### 合法性说明

本DEMO是为了学习研究的目的，所有读者可以参考执行思路和程序代码，但不能用于恶意和非法目的（恶意攻击网站服务器、非法盈利等），如有违者请自行负责。
本DEMO所获取的数据都是仅用于个人学习，并非于恶意抓取数据来攫取不正当竞争的优势，也未用于商业目的牟取不法利益，并非大范围地爬取，同时严格控制爬取的速率、争取不对服务器造成压力，如侵犯当事者（即被抓取的网络主体）的利益，请联系更改或删除。

##### 本DEMO环境配置    
Python：3.6   
Mysql：8.0   
Google Chrome版本：81.0.4044.113   
ChromeDriver：81.0.4044.69   
Jupyter Notebook/ Visual Studio Code   


#### 导入所需的库

In [1]:
import re
import sys
import os
import datetime
import time

import schedule
import docx
import openpyxl
import smtplib
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
from docx.shared import RGBColor
from docx.shared import Inches
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from pptx.util import Inches
from pptx import Presentation
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header
from selenium import webdriver
from multiprocessing import Pool
from loguru import logger

Selenium库是一个自动化测试工具，能够驱动浏览器模拟人的操作，如鼠标单击、键盘输入等。通过Selenium库能够比较容易地获取到网页的源代码，还可以进行网络内容的批量自动下载。  

使用Selenium库之前，要先下载并安装ChromeDriver。它的作用是给Python提供一个模拟浏览器，让Python通过这个模拟浏览器访问网页，并通过Selenium库进行鼠标和键盘等操作，获取到网页真正的源代码。  

1．安装谷歌浏览器并查看版本号；  
2．下载与谷歌浏览器对应版本号的ChromeDriver；  
官方下载地址：https://sites.google.com/a/chromium.org/chromedriver/downloads 若官网访问不了，可用以下的镜像网站下载   
镜像网站：http://npm.taobao.org/mirrors/chromedriver/  
3．ChromeDriver安装；  
下载完ChromeDriver之后，还需要把它安装到Python的安装路径中，让Python能更方便地调用它。  

In [2]:
def selenium(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # 无界面模式
    options.add_argument('lang=zh_CN.UTF-8') # 
    UserAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
    options.add_argument('User-Agent=' + UserAgent)
    prefs = { 
        'profile.default_content_setting_values': {
            'images': 2,
            'permissions.default.stylesheet':2,
            'javascript': 2
        }
    }
    options.add_experimental_option("prefs", prefs) # 优化selenium，禁止加载图片/css/javascript
    browser = webdriver.Chrome(options=options)
    browser.implicitly_wait(10) # 设置隐性等待时间为10s
#     browser.set_page_load_timeout(5)  # 设置页面加载超时
#     browser.set_script_timeout(5) # 设置页面异步js执行超时
    try:
        browser.get(url)
        pagedata = browser.page_source
        browser.quit()
        logger.info('成功访问')
        return pagedata
    except:
        browser.refresh() # 第一次若不能成功访问，则刷新页面
        time.sleep(5)
        pagedata = browser.page_source
        browser.quit()
        logger.info('刷新成功访问')
        return pagedata
    else: # 
        logger.error('无法访问')

正则表达式是非常好用的信息提取工具，它可以灵活、高效地提取文本中的所需信息。  
这里数据提取和数据清洗主要使用以下正则表达式的方法：
1.findall()函数    
2.非贪婪匹配之 (.*?) 用于获取文本A与文本B之间的内容       

3.非贪婪匹配之 .*? 用于代替文本C和文本D之间的所有内容   
4.自动考虑换行的修饰符re.S  
5.sub()函数  

In [3]:
def eastmoney2mysql(company, num):

    url = 'http://so.eastmoney.com/news/s?keyword=%s&pageindex=%s' % (company, num) # 传入所需监控的公司和爬取的页码
    
    data = selenium(url) # selenium数据采集
    p_title = '<div class="news-item"><h3><a href=".*?">(.*?)</a>' # 编写正则表达式提取数据
    p_href = '<div class="news-item"><h3><a href="(.*?)">.*?</a>'
    p_date_text = '<p class="news-desc">(.*?)</p>'

    title = re.findall(p_title,data) # findall()返回的是一个列表
    href = re.findall(p_href,data)
    date = re.findall(p_date_text,data,re.S)
    text = re.findall(p_date_text,data,re.S)
    number = len(title) # 查看标题列表的数量

    if number == 0: # 若返回的标题列表数量为0，则代表无数据，可能为东方财富网无此关键词
        logger.error(company + ' - 第' + str(num) + '页东方财富网爬取失败，无此关键词')
        return False
    else: 
        logger.info(company + ' - 第' + str(num) + '页东方财富网爬取成功')
        
        sql_1 = 'SELECT * FROM test2 WHERE company = "%s"' % company
        fm = Fmysql() # Fmysql类实例化
        data_all = fm.data_from_mysql(sql_1) # 调用Fmysql类提取数据
         # 列表推导式，存放从数据库提取有关该公司的所有标题，为录入数据库去重准备
        title_all = [data_all[j][3] for j in range(len(data_all))]
        
        try:
            for i in range(number):
                title[i] = re.sub('<.*?>', '', title[i]) # 使用正则表达式清洗数据
                date[i] = date[i].split(' ')[0]
                text[i] = re.sub('<.*?>', '', text[i], re.S)
                text[i] = re.sub('\d{4}(.*)\d{2}\s\S', '', text[i], re.S)
                text[i] = re.sub('\n', '', text[i], re.S)
                text[i] = re.sub('\s', '', text[i], re.S)

                print(str(i+1) + '.' + title[i] + ' - '+ date[i])
                print(href[i])
                print(text[i])
                # 数据去重及将数据存入数据库
                if title[i] not in title_all: # 判断数据是否在原数据库中，不在的话才进行数据存储
                    sql_2 = 'INSERT INTO test2(date, company, title, href, text) VALUES("%s", "%s", "%s", "%s", "%s")'\
                                                                    % (date[i], company, title[i], href[i], text[i])
                    fm.execute_to_mysql(sql_2)
            logger.info(company + ' - 第' + str(num) + '页东方财富网录入数据库成功')
        except:
            logger.error(company + ' - 第' + str(num) + '页东方财富网录入数据库失败')

自定义的Fmysql类，方便调用pymysql常用的一些语句  
pymysql官方文档链接：https://pymysql.readthedocs.io/en/latest/

In [4]:
class Fmysql:
    
    def __init__(self, host='localhost', port=3306, user='root', password='xxxxxx', database='xxxx', charset='utf8'):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.charset = charset
        self.conn = None
        self.cur = None
        
    def connect(self): # 连接数据库
        self.conn = pymysql.connect(host=self.host, port=self.port, user=self.user, 
                                        password=self.password, database=self.database, charset=self.charset) # 跟数据库建立连接
        self.cur = self.conn.cursor()

    def close(self): # 关闭数据库
        self.cur.close()
        self.conn.close()
        
    def __execute(self, sql):
        try:
            self.connect()
            count = self.cur.execute(sql)
            self.conn.commit()
        except self.conn.Error:
            logger.error("数据库执行失败！")
            self.conn.rollback()

    def execute_to_mysql(self, sql): # 插入/修改/删除数据
        self.__execute(sql)
        self.close()

    def data_from_mysql(self, sql): # 查询数据内容
        try:
            self.__execute(sql)
            data = self.cur.fetchall()
            self.close()
        except self.conn.Error:
            logger.error("查询数据失败！")
        return data    
    
    def spec_from_mysql(self, sql): # 查询数据表头
        try:
            self.__execute(sql)
            spec = self.cur.description
            self.close()
        except self.conn.Error:
            logger.error("查询表头失败！")
        return spec

获取昨天/今天的日期字符串函数

In [5]:
def get_Ystd():
    # 获取昨天日期的字符串格式的函数
    today = datetime.date.today() #获取今天的日期
    oneday = datetime.timedelta(days=1) #获取一天的日期格式数据
    yesterday = today - oneday
    yesterday_str = yesterday.strftime('%Y-%m-%d') #获取昨天日期的格式化字符串
    return yesterday_str #返回昨天的字符串
def get_Today():
    # 获取今天的日期的字符串格式的函数
    today = datetime.date.today()
    today_str = today.strftime('%Y-%m-%d')
    return today_str

通过pandas的read_sql()方法直接调用mysql导出数据，筛选出某一天的所有公司新闻数据，通过groupby分组统计当天所有公司的新闻数量，再使用matplotlib画图  
matplotlib pyplot subplots的官方文档链接：https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html#matplotlib.pyplot.subplots

In [6]:
def save2picture(date, path):
    # 从mysql读取数据并处理
    db = pymysql.connect(host='localhost', port=3306, user='root', password='xxxxxx', database='xxxx', charset='utf8')
    cursor = db.cursor()
    df = pd.read_sql('SELECT * FROM test2 WHERE date = "%s"' % date, db)
    df = df[['date', 'company']].groupby('company').count().sort_values('date', ascending = False)
    df.reset_index(inplace=True)
    mn=df['date'].min() # 当日所有公司中新闻最少的数量
    mx=df['date'].max() # 当日所有公司中新闻最多的数量

    # Draw plot
    plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
    fig, ax = plt.subplots(figsize=(42,25), dpi= 80)
    ax.hlines(y=df.company.sort_values(ascending = False), xmin=mn, xmax=mx, color='gray', alpha=0.7, linewidth=1, linestyles='dashdot')
    ax.scatter(y=df.company.sort_values(ascending = False), x=df.date, s=75, color='firebrick', alpha=0.6)
    for date, company in enumerate(df.date.sort_values(ascending = True)): 
        ax.text(company+0.5, date, round(company, 1), horizontalalignment='center', verticalalignment='center', fontdict={'size':35})
        
    # Title, Label, Ticks and Ylim
    ax.set_title('今日新闻概览图', fontdict={'size':60})
    ax.set_xlabel('新闻数量', fontdict={'size':35})
    ax.set_yticks(df.company.sort_values(ascending = False))
    ax.set_yticklabels(df.company.str.title(), fontdict={'horizontalalignment': 'right','size':35})
    ax.set_xlim(0, mx+1)
#     plt.show()
    auto_pic = plt.savefig(path, dpi=80)
    return auto_pic

使用openpyxl库，调用Fmysql类将mysql中的数据写入excel  
python-docx的官方文档链接：https://openpyxl.readthedocs.io/en/stable/

In [7]:
def save2excel(date, path):
    # 将数据和字段名写入excel的函数
    excel = openpyxl.Workbook() #新建一个工作薄对象
    sheet = excel.active #激活一个新的sheet
    sheet.title = date #给sheet命名
    
    #生成数据
    sql = 'SELECT * FROM test2 WHERE date = "%s"' % date
    fm = Fmysql()
    data = fm.data_from_mysql(sql) # 生成数据
    spec = fm.spec_from_mysql(sql) # 生成字段名称
    
    #导入数据
    for col in range(len(spec)): #将字段名称循环写入excel第一行，因为字段格式列表里包含列表，每个列表的第一元素才是字段名称
        #row代表行数，column代表列数，value代表单元格输入的值，行数和列数都是从1开始，这点于python不同要注意
        _ = sheet.cell(row = 1, column = col + 1, value = u'%s' % spec[col][0])
    for row in range(len(data)): #将数据循环写入excel的每个单元格中 
        for col in range(len(spec)):
            _ = sheet.cell(row = row + 2, column = col + 1, value = u'%s' % data[row][col]) #因为第一行写了字段名称，所以要从第二行开始写入
    auto_excel = excel.save(path) 
    return auto_excel #返回生成的excel

使用python-docx库，调用Fmysql类将mysql中的数据写入word  
python-docx的官方文档链接：https://python-docx.readthedocs.io/en/latest/index.html

In [8]:
def save2docx(date, company, path):
    # 创建一个空白Word对象，并设置好字体
    word = docx.Document()
    word.styles['Normal'].font.name = u'微软雅黑'  # 可换成word里面任意字体
    word.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')  # 这里也需要同时修改

    # 创建封面    
    p = word.add_picture('E:/Four Jarvis LOGO.png', width=Inches(1.25)) # 创建一个图片
    p = word.add_paragraph()  # 创建一个段落
    p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 居中设置
    p.paragraph_format.space_before = Pt(150)  # 段前距为200，这个是测试出来的
    p.paragraph_format.space_after = Pt(30)  # 段后距为40，这个也是测试出来的
    run = p.add_run('Four Jarvis数据分析报告')  # 在段落里添加内容
    font = run.font  # 设置字体
    font.color.rgb = RGBColor(17, 17, 17)  # 颜色设置，这里是用RGB颜色
    font.size = Pt(36)  # 字体大小设置，和word里面的字号相对应

    p = word.add_paragraph()  # 新建一个段落
    p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = p.add_run(date)  # 在段落中输入当天日期
    font = run.font
    font.color.rgb = RGBColor(17, 17, 17)
    font.size = Pt(26)
    
    for x in company:
        # 添加分页符
        word.add_page_break()# 添加分页符
        
        sql = 'SELECT * FROM test2 WHERE company = "%s" AND date = "%s"' % (x, date)
        fm = Fmysql()
        data = fm.data_from_mysql(sql) # 生成数据
        num = len(data)
        
        if num != 0: 
            # 设置正文标题
            p = word.add_paragraph()
            p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 段落文字居中设置
            run = p.add_run(x + '舆情报告')
            run.font.color.rgb = RGBColor(17, 17, 17)  # 字体颜色设置
            run.font.size = Pt(22)  # 字体大小设置

            # 编写正文内容之引言
            p = word.add_paragraph()  # 添加新段落
            p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # 两端对齐
            p.paragraph_format.first_line_indent = Inches(0.2)  # 控制首行缩进
            introduction = '本次舆情监控目标:%s，当天共爬取东方财富网内相关新闻%s篇，具体新闻如下：' % (x, num)
            p.add_run(introduction).bold = True # 加粗

            # 编写正文内容之具体新闻内容
            for i in range(num):
                p = word.add_paragraph()  # 添加新段落
                p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # 设置两端对齐
                p.add_run(str(i + 1) + '. ' + data[i][3])  # 提取新闻标题

            # 编写正文内容之表格添加
            tb = word.add_table(rows=num + 1, cols=2, style='Light Grid')
            tb.cell(0, 0).text = '新闻标题'
            tb.cell(0, 1).text = '新闻摘要'

            for i in range(num):
                tb.cell(i+1, 0).text = data[i][3] + data[i][4]  # 提取新闻标题 + 新闻链接
                tb.cell(i+1, 1).text = data[i][5]  # 提取新闻摘要
        else:
            # 设置正文标题
            p = word.add_paragraph()
            p.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 段落文字居中设置
            run = p.add_run(x + '舆情报告')
            run.font.color.rgb = RGBColor(17, 17, 17)  # 字体颜色设置
            run.font.size = Pt(22)  # 字体大小设置
            
            p = word.add_paragraph()  # 添加新段落
            p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # 两端对齐
            p.paragraph_format.first_line_indent = Inches(0.4)  # 这个控制首行缩进，
            introduction = '本次舆情监控目标:%s，当天东方财富网内无该公司相关新闻' % x
            p.add_run(introduction).bold = True # 加粗
            
    auto_docx = word.save(path)
    return auto_docx

使用python-pptx库，调用Fmysql类将mysql中的数据写入ppt  
python-pptx的官方文档链接：https://python-pptx.readthedocs.io/en/latest/

In [9]:
def save2pptx(date, company, path):

    prs = Presentation('D:/python-bit/Four Jarvis/Demo1/model.pptx') # 使用预先创建好的PPT模板,也可以直接生成
    slide_1 = prs.slides[0] # 创建ppt封面 第一页
    title = slide_1.shapes[0]
    subtitle = slide_1.shapes[1]
    title.text = "Four Jarvis 数据分析报告"
    subtitle.text = date
    
    img_path = 'D:/python-bit/Four Jarvis/Demo1/Four Jarvis Auto Demo1 %s.jpg' % date # 图片储存路径
    slide_2 = prs.slides[1] # 创建ppt 第二页 插入图片概览
    pic = slide_2.shapes.add_picture(img_path, left=Inches(0), top=Inches(0), height=Inches(8)) 
    
    for x in company:        
        #templateStyleNum = len(prs.slide_layouts) # 获取模板个数
        oneSlide = prs.slides.add_slide(prs.slide_layouts[0]) # 按照第1个模板创建 一张幻灯片 
        sql = 'SELECT * FROM test2 WHERE company = "%s" AND date = "%s"' % (x, date)
        fm = Fmysql()
        data = fm.data_from_mysql(sql) # 生成数据
        num = len(data)
        
        if num != 0:        
            body_shapes = oneSlide.shapes.placeholders # 获取模板可填充的所有位置
            table_placeholder = oneSlide.shapes[2]
            
            for index, body_shape in enumerate(body_shapes):
                if index == 0:
                    body_shape.text = '%s舆情报告' % x
                elif index == 1:
                    body_shape.text = '本次舆情监控目标:%s，当天共爬取东方财富网内相关新闻%s篇，具体新闻如下：'% (x, num)
                elif index == 2:
                    rows, cols = num + 1, 2
                    tb = table_placeholder.insert_table(rows, cols).table  # 添加表格，并取表格类

                    tb.columns[0].width = Inches(4.5)  # 第一列宽度
                    tb.columns[1].width = Inches(8.8)  # 第二列宽度
                    tb.cell(0, 0).text = '新闻标题'
                    tb.cell(0, 1).text = '新闻摘要'

                    for i in range(num):
                        tb.cell(i+1, 0).text = data[i][3] + data[i][4]  # 提取新闻标题 + 新闻链接               
                        tb.cell(i+1, 1).text = data[i][5]  # 提取新闻摘要                        
        else:
            body_shapes = oneSlide.shapes.placeholders
            for index, body_shape in enumerate(body_shapes):
                if index == 0:
                    body_shape.text = '%s舆情报告' % x
                elif index == 1:
                    body_shape.text = '本次舆情监控目标:%s，当天东方财富网内无该公司相关新闻!' % x
    auto_pptx = prs.save(path)
    return auto_pptx

SMTP（Simple Mail Transfer Protocol）即简单邮件传输协议,它是一组用于由源地址到目的地址传送邮件的规则，由它来控制信件的中转方式。

Python的smtplib提供了一种很方便的途径发送电子邮件，它对SMTP协议进行了简单的封装。

In [10]:
def create_email(email_from, email_to, email_subject, email_text, attach_path, attach_name):
    # 输入发件人昵称、收件人昵称、主题，正文，附件地址,附件名称生成一封邮件
    message = MIMEMultipart() # 生成一个空的带附件的邮件实例
    message.attach(MIMEText(email_text, 'plain', 'utf-8')) # 将正文以text的形式插入邮件中
    message['From'] = Header(email_from, 'utf-8') # 生成发件人名称（这个跟发送的邮件没有关系）
    message['To'] = Header(email_to, 'utf-8') # 生成收件人名称（这个跟接收的邮件也没有关系）
    message['Subject'] = Header(email_subject, 'utf-8') # 生成邮件主题                 
    for name in attach_name:
        if os.path.isfile(attach_path + '/' + name):
            # 构造附件
            att = MIMEText(open(attach_path + '/' + name, 'rb').read(), 'base64', 'utf-8')
            att["Content-Type"] = 'application/octet-stream'
            att.add_header("Content-Disposition", "attachment", filename=("gbk", "", name))
            message.attach(att)
    return message # 返回邮件

In [11]:
def send_email(sender, password, receiver, message):
    # 输入发件人邮箱、密码、收件人邮箱、邮件内容发送邮件的函数
    try:
        server = smtplib.SMTP_SSL("smtp.qq.com", 465)  # 发件人邮箱中的SMTP服务器
        server.ehlo()
        server.login(sender, password)  # 登录qq邮箱
        #发送邮件
        server.sendmail(sender, receiver, message.as_string())  # 括号中对应的是发件人邮箱账号、收件人邮箱账号（是一个列表）、邮件内容
        print("邮件发送成功")
        server.quit()  # 关闭连接
    except Exception:
        print(traceback.print_exc())
        print("邮件发送失败")

通过Schedule库实现每天定时发送/或者每小时...但是时间周期设置时需要考虑这次任务还未完成，就开始执行下次的情况...也可以为为每个任务创建一个线程，让任务在线程中运行，从而并发工作。

In [13]:
def main_by_schedule(main):
#     schedule.every(5).minutes.do(main)              #每5分钟执行一次
#     schedule.every().hour.do(main)                   #每小时执行一次  
    schedule.every().day.at("23:43").do(main)        #每天23:43执行一次  
#     schedule.every().monday.do(main)                 #每周一执行一次
#     schedule.every().monday.at("22:45").do(main)  #每周一22:45执行一次
   
    while True:
        schedule.run_pending()
        time.sleep(7) # 这里时间设置是为schedule函数停顿一下，可改为1s，while True会一直循环执行

因为selenium采集数据很慢，所以本Demo使用multiprocessing.Pool类来简单实现程序并行化，设定同时运行的进程的数量为3，并采用apply_async，异步变体更适合运行并行任务，不需要关心进程返回结果的顺序，但是多进程对内存、CPU和网络带宽的要求更高...

在使用前首先要了解一下什么是并行/并发/异步/协程/线程/进程/GIL/I/O操作/队列...再去考虑使用多线程/多进程

简而言之，multiprocessing.Pool类提供了使程序并行化的简单方法。它用来管理一个工作进程池，并被定义为给定的任务提供各种并行执行的方法。  

在Python中，线程模块提供了一个基于线程并行化的高级接口。为避免竞争条件，Python采用了一种称为全局解释器锁（GIL）的机制。当线程执行一组代码时，需要获取全局锁。此锁确保在Python解释器环境中每次只执行一个线程。GIL的缺点是不能充分利用多处理器。

multiprocessing为了解决GIL问题，提供了一个简单的方法来并行Python程序。它通过使用子进程，而不使用线程来避免GIL。在这个模块中，使用两个通信信道、一个队列类和一个管道函数来支持进程间的数据交换。该模块还提供了一些其他有用的功能，如管理和代理对象。管理对象使用multiprocessing.Manager()来创建。它控制管理Python对象的服务器进程。管理对象还允许其他进程使用代理来操作这些Python对象。Python文档中有如何让这些功能很好工作的例子。

multiprocessing.Pool官方文档链接：https://docs.python.org/3/library/multiprocessing.html#module-multiprocessing.pool

main()函数是主程序，这里配置好所需监控的公司列表、每次爬取的页面数量、文件存储路径、日期默认为当天、文件命名、以及邮件信息，均可根据自己的需求修改

In [ ]:
def main():
    
    # 监控目标公司列表，仅作演示
    companies = ['高伟达', '飞天诚信', '朗科科技', '永太科技', '海螺水泥', '瑞幸咖啡', '腾讯控股', '腾讯', '分众传媒',\
            '阿里巴巴', '中国联通', 'TCL科技', '中公教育', '华北制药', '亿帆医药', '圣达生物', '振华股份', '三一重工',\
            '海天味业', '贵州茅台', '美的集团', '中国平安', '华润三九', '索菲亚', '格力电器', '白云机场', '海康威视']
    page = 2 # 每次爬取页面数量
    path = 'D:/python-bit/Four Jarvis/Demo1' # 文件存储路径
    today = get_Today() # 或者自定义日期'xxxx-xx-xx'
    logger.add(f'Four Jarvis Auto Demo1 {today}.log') # 日志记录
    
    print('爬虫程序开始执行：')
    start = time.time()    
    #单机串行实现
#     for num in range(page):
#         for company in companies:
#             eastmoney2mysql(company, num + 1)
            
#     多进程实现数据采集和存储
    pool = Pool(3) # 创建进程池
    for num in range(page):
        for company in companies:
            pool.apply_async(eastmoney2mysql(company, num + 1))
    pool.close()
    pool.join()

    end = time.time()
    print('爬虫程序执行结束！总用时：' + str(end-start))
    
    #AUTO_PICTURE  
    my_file_name1 = 'Four Jarvis Auto Demo1 %s.jpg' % today
    file_path1 = path + '/' + my_file_name1
    save2picture(today, file_path1) # 生成picture    
    
    #AUTO_EXCEL  
    my_file_name2 = 'Four Jarvis Auto Demo1 %s.xlsx' % today
    file_path2 = path + '/' + my_file_name2
    save2excel(today, file_path2) # 生成excel
    
    #AUTO_WORD
    my_file_name3 = 'Four Jarvis Auto Demo1 %s.docx' % today
    file_path3 = path + '/' + my_file_name3
    save2docx(today, companies, file_path3) # 生成docx
    
    #AUTO_PPT
    my_file_name4 = 'Four Jarvis Auto Demo1 %s.pptx' % today
    file_path4 = path + '/' + my_file_name4
    save2pptx(today, companies, file_path4) # 生成pptx

    #AUTO_MAIL
    my_email_from = 'Four Jarvis'
    my_email_to = 'User'
    my_email_subject = 'Four Jarvis Auto Demo1 ' + today # 邮件标题
    my_email_text = "Dear all,\n\t\t附件为今天的数据分析报告，请查收！\n\nFour Jarvis " # 邮件正文
    
    my_attach_name = [my_file_name1, my_file_name2, my_file_name3, my_file_name4] # 附件名称
    my_message = create_email(my_email_from, my_email_to, my_email_subject, my_email_text, path, my_attach_name)  # 生成邮件
    my_sender = 'xxxxxxxxx@qq.com'
    my_password = 'xxxxxxxxxxxxxxxxx' # QQ邮箱的SMTP授权码
    my_receiver = ['xxxxxxxxx@qq.com', 'xxxxxxxxx@qq.com', 'xxxxxxxxx@qq.com'] # 接收人邮箱列表
    send_email(my_sender, my_password, my_receiver, message) # 发送邮件

if __name__ == '__main__':
    main_by_schedule(main) # 定时执行

爬虫程序开始执行：


2020-04-22 23:43:15.730 | INFO     | __main__:selenium:25 - 成功访问
2020-04-22 23:43:15.736 | INFO     | __main__:eastmoney2mysql:20 - 高伟达 - 第1页东方财富网爬取成功
2020-04-22 23:43:15.760 | INFO     | __main__:eastmoney2mysql:44 - 高伟达 - 第1页东方财富网录入数据库成功


1.数字货币内测风起 央行降温难阻市场热捧 - 2020-04-21
http://finance.eastmoney.com/a/202004211462269048.html
4月20日开盘，数字货币概念股再次走强，飞天诚信(300386。)拉升封板，数字认证(300579)、四方精创(300468)、高伟达(300465)等快速跟涨。近日，央行数字货币(DC/EP)在内部测试的消息不断，引发了外界对于数字货币即将落地的猜想，相关板块股票...
2.投资者安利“数字货币” 上市公司纷纷澄清：我没有！ - 2020-04-21
http://finance.eastmoney.com/a/202004211462007731.html
铁打的A股，流水的概念。4月20日午后，数字货币概念股股价表现突出，这也再度引发了投资者对于这类概念股的关注度。“恨公司不数字货币”的现象，在投资者互动平台再度上演。《证券日报》记者在深交所互动平台，以“数字货币”为关键词查询得知，在最近一周的时间里，关于...
3.高伟达4月21日快速反弹 - 2020-04-21
http://stock.eastmoney.com/a/202004211461619354.html
以下是高伟达在北京时间4月21日09:38分盘口异动快照：4月21日，高伟达盘中快速反弹，5分钟内涨幅超过2%，截至9点38分，报14.34元，成交1.12亿元，换手率1.77%。分笔09:38:1514.34305↓09:38:1214.32219↓09:38:0914.334...
4.高伟达4月21日盘中跌幅达5% - 2020-04-21
http://stock.eastmoney.com/a/202004211461587410.html
以下是高伟达在北京时间4月21日09:33分盘口异动快照：4月21日，高伟达盘中跌幅达5%，截至9点33分，报14.05元，成交6534.58万元，换手率1.02%。分笔09:33:4514.05660↓09:33:4214.08381↑09:33:3914.08904↑09:3...
5.高伟达4月20日盘中涨幅达5% - 2020-04-20
http://stock.eastmoney.com/a/202004201460293494.html
以下是高伟达在北

2020-04-22 23:43:25.262 | INFO     | __main__:selenium:25 - 成功访问
2020-04-22 23:43:25.264 | INFO     | __main__:eastmoney2mysql:20 - 飞天诚信 - 第1页东方财富网爬取成功
2020-04-22 23:43:25.275 | INFO     | __main__:eastmoney2mysql:44 - 飞天诚信 - 第1页东方财富网录入数据库成功


1.飞天诚信4月22日快速上涨 - 2020-04-22
http://stock.eastmoney.com/a/202004221463431076.html
以下是飞天诚信在北京时间4月22日13:32分盘口异动快照：4月22日，飞天诚信盘中快速上涨，5分钟内涨幅超过2%，截至13点32分，报18.21元，成交3.69亿元，换手率9.83%。分笔13:32:0018.21152↑13:31:5718.20176↓13:31:5418....
2.飞天诚信4月22日快速反弹 - 2020-04-22
http://stock.eastmoney.com/a/202004221463344196.html
以下是飞天诚信在北京时间4月22日10:25分盘口异动快照：4月22日，飞天诚信盘中快速反弹，5分钟内涨幅超过2%，截至10点25分，报17.5元，成交2.31亿元，换手率6.24%。分笔10:25:0017.51437↑10:24:5717.5027↓10:24:5417.50...
3.飞天诚信4月22日盘中跌幅达5% - 2020-04-22
http://stock.eastmoney.com/a/202004221462808519.html
以下是飞天诚信在北京时间4月22日09:33分盘口异动快照：4月22日，飞天诚信盘中跌幅达5%，截至9点33分，报16.78元，成交5943.88万元，换手率1.60%。分笔09:33:0016.78316↓09:32:5716.82247↓09:32:5416.85782↑09...
4.数字货币内测风起 央行降温难阻市场热捧 - 2020-04-21
http://finance.eastmoney.com/a/202004211462269048.html
4月20日开盘，数字货币概念股再次走强，飞天诚信(300386。)拉升封板，数字认证(300579)、四方精创(300468)、高伟达(300465)等快速跟涨。近日，央行数字货币(DC/EP)在内部测试的消息不断，引发了外界对于数字货币即将落地的猜想，相关板块股票...
5.三天两板飞天诚信：未参与央行数字货币试点 拟圈钱8.5亿 - 2020-04-21
http://finance.eastmoney.com/a/20200421146204093

...

1.格力电器：融资净偿还2118.74万元，融资余额54.24亿元（04-17） - 2020-04-20
http://stock.eastmoney.com/a/202004201459963091.html
格力电器融资融券信息显示，2020年4月17日融资净偿还2118.74万元；融资余额54.24亿元，较前一日下降0.39%。融资方面，当日融资买入1.84亿元，融资偿还2.06亿元，融资净偿还2118.74万元。融券方面，融券卖出12.86万股，融券偿还44.99万股，融券余量6...
2.格力电器(000651)融资融券信息(04-17) - 2020-04-20
http://stock.eastmoney.com/a/202004201459796215.html
格力电器(000651)2020-04-17融资融券信息显示，格力电器融资余额5,423,615,707元，融券余额34,266,925元，融资买入额184,397,545元，融资偿还额205,584,955元，融资净买额-21,187,410元，融券余量626,452股，融券卖...
3.【大佬去哪儿】董明珠发表“鲍鱼论” 何小鹏“没想到造车这么花钱” - 2020-04-19
http://finance.eastmoney.com/a/202004191459466320.html
大佬成功的经验不可复制，但跟着大佬或许可以少走弯路。本周，中新经纬继续推出《大佬去哪儿》小栏目，回顾过去一周大佬的行踪、言论。接下来，让我们跟上大佬的脚步，一起出发……何小鹏：我知道造车很花钱，但没想到这么花钱“我知道很花钱，但没想到这么花钱。”近日，小鹏...
4.格力自主研发的“猎手”空气净化器亮相央视 - 2020-04-19
http://finance.eastmoney.com/a/202004191459451557.html
拥有自主科技和自主创新的格力“猎手”空气净化器，作为有效抗击病毒的空气净化产品，目前已销往40个国家。根据多家权威实验室检测数据表明，格力杀病毒空气净化器，1小时可杀灭空气中99%的病毒。
5.奥克斯深陷“围城”：何解“低端之痛”？ - 2020-04-18
http://finance.eastmoney.com/a/202004181459123734.html


2020-04-22 23:58:10.314 | INFO     | __main__:selenium:25 - 成功访问
2020-04-22 23:58:10.318 | INFO     | __main__:eastmoney2mysql:20 - 白云机场 - 第2页东方财富网爬取成功
2020-04-22 23:58:10.340 | INFO     | __main__:eastmoney2mysql:44 - 白云机场 - 第2页东方财富网录入数据库成功


1.白云机场：连续5日融资净偿还累计1027.85万元（04-15） - 2020-04-16
http://stock.eastmoney.com/a/202004161456130241.html
白云机场融资融券信息显示，2020年4月15日融资净偿还235.46万元；融资余额2.07亿元，较前一日下降1.12%。融资方面，当日融资买入414.73万元，融资偿还650.19万元，融资净偿还235.46万元，连续5日净偿还累计1027.85万元。融券方面，融券卖出1.64万...
2.白云机场(600004)融资融券信息(04-14) - 2020-04-15
http://stock.eastmoney.com/a/202004151454737328.html
白云机场(600004)2020-04-14融资融券信息显示，白云机场融资余额209,383,252元，融券余额6,141,760元，融资买入额2,091,008元，融资偿还额4,758,201元，融资净买额-2,667,193元，融券余量451,600股，融券卖出量0股，融券偿...
3.白云机场：连续4日融资净偿还累计792.38万元（04-14） - 2020-04-15
http://stock.eastmoney.com/a/202004151454596098.html
白云机场融资融券信息显示，2020年4月14日融资净偿还266.72万元；融资余额2.09亿元，较前一日下降1.26%。融资方面，当日融资买入209.1万元，融资偿还475.82万元，融资净偿还266.72万元，连续4日净偿还累计792.38万元。融券方面，融券卖出0股，融券偿还...
4.白云机场(600004)融资融券信息(04-13) - 2020-04-14
http://stock.eastmoney.com/a/202004141453138939.html
白云机场(600004)2020-04-13融资融券信息显示，白云机场融资余额212,050,445元，融券余额6,724,032元，融资买入额3,089,694元，融资偿还额3,781,702元，融资净买额-692,008元，融券余量500,300股，融券卖出量54,600股，...
5.白云机场：连续3日融资净偿还累计525.67万元（04-13） -

2020-04-22 23:58:18.798 | INFO     | __main__:selenium:25 - 成功访问
2020-04-22 23:58:18.802 | INFO     | __main__:eastmoney2mysql:20 - 海康威视 - 第2页东方财富网爬取成功
2020-04-22 23:58:18.817 | INFO     | __main__:eastmoney2mysql:44 - 海康威视 - 第2页东方财富网录入数据库成功


1.海康威视现1笔大宗交易 机构卖出1.69亿元 - 2020-04-20
http://finance.eastmoney.com/a/202004201460678553.html
海康威视(002415)今日收涨2.08%，报30.87元。盘后数据显示，该股当日出现1笔大宗交易，成交价为收盘价，成交金额1.69亿元。卖出席位为机构专用席位，买入席位为北京高华证券北京金融大街证券营业部。（文章来源：e公司）
2.海康威视04月20日发生1笔大宗交易 成交1.69亿元 - 2020-04-20
http://stock.eastmoney.com/a/202004201460604474.html
2020年04月20日，海康威视发生了1笔大宗交易，总成交数量为546.74万股，总成交金额为1.69亿元，成交价格为30.87元，当日收盘报价30.87元，涨幅2.08%，成交金额13.42亿元；大宗交易成交金额占当日成交金额12.58%，溢价0.00%。大宗交易详情如下表...
3.真是“好同学”！河南首富为这所大学再捐8个亿 其身家已超他们！ - 2020-04-20
http://finance.eastmoney.com/a/202004201460556155.html
4月19日晚间，牧原股份发布公告，称为促进西湖大学教育科研事业的发展，支持西湖大学培养高素质人才、建设世界一流大学，公司董事长秦英林拟自愿向杭州市西湖教育基金会再次无偿捐赠其所持牧原股份市值8亿元人民币对应的数量股票，具体捐赠股票数量以非交易过户时的股票价格确定。...
4.海康威视：融资净偿还7421.17万元，两市排名第13（04-17） - 2020-04-20
http://stock.eastmoney.com/a/202004201459988195.html
海康威视融资融券信息显示，2020年4月17日融资净偿还7421.17万元；融资余额17.21亿元，较前一日下降4.13%。融资方面，当日融资买入8093.16万元，融资偿还1.55亿元，融资净偿还7421.17万元，净偿还额两市排名第13，连续5日净偿还累计2.7亿元。融券方面...
5.海康威视(002415)融资融券信息(04-17) - 2020-04-20
http://stock.eastmoney.com/a